# TODO:

1. Find some good candidates for binary classification including ensembles
    1. 5 different models
2. Write a pipeline for testing and analysis of the results
3. Create code documentation :)
4. Maybe extract the functions to separate .py to make the notebook cleaner
3. Write the report
4. Prepare the presentation

# Imports and Consts

In [40]:
import pickle
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis,
)
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools
from enum import Enum

# Functions

## Generate parameters

- takes any number of arrays and returns an array of dictionaries with keys as array names and values, all possible combinations

In [4]:
# Get name of a variable as string
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

In [5]:
def generateParameters(arrays):
    allCombinations = list(itertools.product(*arrays))
    allCombinations = [list(elem) for elem in allCombinations]
    arrayNames = []
    for array in arrays:
        arrayNames.append(namestr(array, globals())[0])
    return [dict(zip(arrayNames, value)) for value in allCombinations]

- example usage

In [6]:
arr1 = [1, 2, 7]
arr2 = [3, 4, 8]
generateParameters([arr1, arr2])

[{'arr1': 1, 'arr2': 3},
 {'arr1': 1, 'arr2': 4},
 {'arr1': 1, 'arr2': 8},
 {'arr1': 2, 'arr2': 3},
 {'arr1': 2, 'arr2': 4},
 {'arr1': 2, 'arr2': 8},
 {'arr1': 7, 'arr2': 3},
 {'arr1': 7, 'arr2': 4},
 {'arr1': 7, 'arr2': 8}]

## Experiment process

Scoring function, based on which, the best model is selected. Score is calculated according to task description: +10 points for each correctly classified positive class, -200 points for each feature used

In [25]:
def getScore(y_true, y_pred, featuresUsed):
    score = 0
    correct = 0
    for i, y in enumerate(y_true):
        if y == y_pred[i]:
            correct += 1
    score = 10 * correct - 200 * featuresUsed
    return score

Main experiment loop, each chosen classifier is tested with different methods of feature selection and different parameters

In [26]:
def performExperiment(X_train, y_train, X_test, y_test, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    finalResult = getScore(y_test, y_pred, len(X_train[0]))
    # Take only 1000 of the highest class 1 probabilities
    # TODO: Turn it into a dataframe and keep track of the indexes
    # finalResult = np.sort(result[:, 1])[::-1][:1000]

    return finalResult

## Model helpers

- definition of available models and feature selection methods

- getters for models and feature selection methods

In [27]:
class ModelType(Enum):
    LDA = 0
    QDA = 1
    DecisionTree = 2

In [28]:
def getModel(modelType, arguments):
    match modelType:
        case ModelType.LDA:
            return LinearDiscriminantAnalysis(**arguments)
        case ModelType.QDA:
            return QuadraticDiscriminantAnalysis(**arguments)
        case ModelType.DecisionTree:
            return DecisionTreeClassifier(**arguments)

In [29]:
class FeatureSelectorType(Enum):
    NoFeatureSelection = 0

In [30]:
class NoFeatureSelection:
    def fit(X):
        return X

In [31]:
def getFeatureSelector(selectorType, arguments):
    match selectorType:
        case FeatureSelectorType.NoFeatureSelection:
            return NoFeatureSelection(**arguments)

# Load Data

load the training and test data in a format specified by task description

In [32]:
with open("./Dataset/x_test.txt") as file:
    X_test = [[float(digit) for digit in line.split()] for line in file]


with open("./Dataset/x_train.txt") as file:
    X_train = [[float(digit) for digit in line.split()] for line in file]


with open("./Dataset/y_train.txt") as file:
    y_train = [[float(digit) for digit in line.split()] for line in file]

In the X_test there are 500 features and 5000 observations, y_train contains 5000 values, X_train contains 500 features and 5000 observations

In [33]:
print("X_test datapoints:", len(X_test), " features:", len(X_test[0]))
print("X_train datapoints:", len(X_train), " features:", len(X_train[0]))
print("y_train datapoints:", len(y_train))

X_test datapoints: 5000  features: 500
X_train datapoints: 5000  features: 500
y_train datapoints: 5000


# Experiment

## Parameters

In [34]:
# TODO: Create parameter arrays

# LDA
solver = ["svd"]
ldaParameters = generateParameters([solver])


models = [{"model": ModelType.LDA, "parameters": ldaParameters}]

In [35]:
featureSelectors = [
    {"model": FeatureSelectorType.NoFeatureSelection, "parameters": [{}]}
]

## Main experiment loop

1. for now just do one train test split
2. For each of the models:
    1. For each of the feature selectors:
        1. Calculate the score of the given combination of model+feature selector based on the +10 for each correct prediction and -200 for each feature used.


## Conducting the experiment

In [42]:
%%time
results=[]
for model in models:
    for parameters in model['parameters']:
        X_split_train, X_split_test, y_split_train, y_split_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)
        
        result = performExperiment(X_train=X_split_train, y_train=y_split_train, X_test=X_split_test, 
        y_test=y_split_test, model=getModel(model['model'], parameters))

        results.append([result,model['model'].name,parameters])
with open("./Results/results", "wb") as f:
    pickle.dump(results, f)

c:\Users\Patryk\Repos\advml-proj-2\.venv\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: total: 4.47 s
Wall time: 1.01 s


In [43]:
resultsDf = pd.DataFrame(results, columns=["score", "model", "parameters"])

In [52]:
resultsDf

,score,model,parameters
0,-91740,LDA,{'solver': 'svd'}


# Results

In [21]:
with open(r"./Results/results", "rb") as input_file:
    results = pickle.load(input_file)